In [1]:
import os

os.environ["DGLBACKEND"] = "pytorch"  # use PyTorch as backend
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
dataset = dgl.data.CoraGraphDataset()
print(f"Number of categories: {dataset.num_classes}")

Extracting file to C:\Users\Atikin\.dgl\cora_v2_d697a464
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


In [3]:
g = dataset[0]
g.ndata["feat"].shape

torch.Size([2708, 1433])

In [4]:
from dgl.nn import GraphConv


class GCN(nn.Module):  # nn.Module - PyTorch parent class for registering all layers, parameters, etc. inside the backing
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)  # a graph convolution layer in which we specify the dimension of incoming and outgoing elements. In our case, this is a vector of words
        self.conv2 = GraphConv(h_feats, num_classes)  # the last layer, at the output we get dimension = 7 - the number of classes in the graph

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


# Create the model with given dimensions
# 16 - number of parameters on the middle layer
model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)

In [5]:
model

GCN(
  (conv1): GraphConv(in=1433, out=16, normalization=both, activation=None)
  (conv2): GraphConv(in=16, out=7, normalization=both, activation=None)
)

#### Loop for model training

In [6]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(100):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                f"In epoch {e}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f})"
            )


train(g, model)

In epoch 0, loss: 1.946, val acc: 0.136 (best 0.136), test acc: 0.125 (best 0.125)
In epoch 5, loss: 1.904, val acc: 0.534 (best 0.594), test acc: 0.529 (best 0.630)
In epoch 10, loss: 1.835, val acc: 0.654 (best 0.662), test acc: 0.684 (best 0.691)
In epoch 15, loss: 1.740, val acc: 0.624 (best 0.662), test acc: 0.637 (best 0.691)
In epoch 20, loss: 1.620, val acc: 0.668 (best 0.668), test acc: 0.703 (best 0.703)
In epoch 25, loss: 1.478, val acc: 0.700 (best 0.702), test acc: 0.729 (best 0.728)
In epoch 30, loss: 1.318, val acc: 0.694 (best 0.704), test acc: 0.734 (best 0.727)
In epoch 35, loss: 1.149, val acc: 0.708 (best 0.708), test acc: 0.747 (best 0.747)
In epoch 40, loss: 0.977, val acc: 0.732 (best 0.732), test acc: 0.748 (best 0.748)
In epoch 45, loss: 0.813, val acc: 0.746 (best 0.746), test acc: 0.752 (best 0.752)
In epoch 50, loss: 0.664, val acc: 0.756 (best 0.756), test acc: 0.756 (best 0.756)
In epoch 55, loss: 0.535, val acc: 0.758 (best 0.758), test acc: 0.763 (best 0

Let's see what accuracy we got in the end

In [7]:
model.eval()
features = g.ndata["feat"]
pred = model(g, features).argmax(dim=1)
print(pred)
correct = (pred[g.ndata['test_mask']] == g.ndata["label"][g.ndata["test_mask"]]).sum()
acc = int(correct) / int(g.ndata['test_mask'].sum())
print(f'Accuracy: {acc:.4f}')

tensor([3, 4, 4,  ..., 0, 4, 3])
Accuracy: 0.7790


Now we save the entire model to a file. Since the model uses PyTorch as a back-up, saving occurs through PyTorch functions

In [8]:
torch.save(model, './models/model.pt')

#### Now let’s load the model and make an inference

In [11]:
model_inf = torch.load('./models/model.pt')
model_inf.eval()

GCN(
  (conv1): GraphConv(in=1433, out=16, normalization=both, activation=None)
  (conv2): GraphConv(in=16, out=7, normalization=both, activation=None)
)

In [12]:
features = g.ndata["feat"]
pred = model_inf(g, features).argmax(dim=1)
print(pred)
correct = (pred[g.ndata['test_mask']] == g.ndata["label"][g.ndata["test_mask"]]).sum()
acc = int(correct) / int(g.ndata['test_mask'].sum())
print(f'Accuracy: {acc:.4f}')

tensor([3, 4, 4,  ..., 0, 4, 3])
Accuracy: 0.7790
